import Basic Module

In [ ]:
import os
import torch
import numpy as np
from pathlib import Path
import time
from datetime import datetime, timedelta
from tqdm import tqdm
# from tqdm.notebook import tqdm
# import tqdm as tqdm_module
# tqdm_module.tqdm = tqdm  # 전역 tqdm 교체
import importlib
import json

# Utils import (모듈화)
from utils import create_dataloaders, CDMetrics, get_loss_fn, CDTrainer



CUDA(GPU) 확인

In [2]:
# 단일 GPU 사용
GPU_ID = 0
os.environ['CUDA_VISIBLE_DEVICES'] = str(GPU_ID)
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using GPU: {GPU_ID}")
print(f"Device: {DEVICE}")

# 멀티 GPU 사용 
# # GPU_IDS = [0, 1, 2, 3]  # 사용할 GPU 리스트
# os.environ['CUDA_VISIBLE_DEVICES'] = ','.join(map(str, GPU_IDS))
# DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# USE_MULTI_GPU = len(GPU_IDS) > 1 and torch.cuda.device_count() > 1
# print(f"Using GPUs: {GPU_IDS}")
# print(f"Available GPU count: {torch.cuda.device_count()}")
# if USE_MULTI_GPU:
#     BATCH_SIZE = BATCH_SIZE * len(GPU_IDS)  # 멀티 GPU시 배치 크기 조정
#     print(f"Adjusted batch size for multi-GPU: {BATCH_SIZE}")
# 시드 설정 (재현가능성)

Using GPU: 0
Device: cuda


데이터셋 & 모델 리스트

In [3]:
DATASET_ROOT = "./dataset"  # 심볼릭 링크된 데이터셋 루트 폴더
DATASET_LIST = [
    'LEVIR-CD+',
    'WHU-CD',
    'CLCD',
    'CaBuAr-CD',
    'S2Looking-CD',
    'SEN1Floods11-CD'
]

MODEL_LIST = [
    'A2Net',
    'Changer',
    'Change3D',
    'STRobustNet',
    'USSFC-Net'
    # 'ChangeMamba',
    # 'CDMamba',
    # 'ChangeCLIP',
    # 'EATDER'
]

시드 설정

In [4]:
SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)

##### 고정 메트릭 설정

In [5]:
# 이미지 설정
IMG_SIZE = 256 
IN_CHANNELS = 3  # RGB
OUT_CHANNELS = 1  # Binary change detection

# 학습 설정
BATCH_SIZE = 64
NUM_WORKERS = 4
MAX_ITERATIONS = 100000  # 데이터셋 크기와 무관하게 고정


##### 모델 & 데이터셋 설정

In [6]:
# 실험할 데이터셋 선택
test_dataset = 'LEVIR-CD+'  # LEVIR-CD+
# test_dataset = DATASET_LIST[0]  # LEVIR-CD+

# 실험할 모델 선택
test_model = 'Change3D'

# True: 최소 구현, False: 전체 구현
use_base = True  

print(f"Test dataset: {test_dataset}")
if test_model not in MODEL_LIST:
    raise ValueError(f"Model {test_model} not in MODEL_LIST. Choose from: {MODEL_LIST}")
print(f"Test model: {test_model}")
print(f"Using base version: {use_base}")

Test dataset: LEVIR-CD+
Test model: Change3D
Using base version: True


모델 동적 import

In [7]:
def get_model_class(model_name, use_base=False):
    """모델 동적 import - 자동 경로 생성"""
    
    # 모델명을 소문자로 변환
    model_name_lower = model_name.lower()
    
    # base/full에 따른 경로 및 클래스명 생성
    if use_base:
        module_path = f'models.{model_name_lower}_base'
        class_name = f'{model_name}Base'
    else:
        module_path = f'models.{model_name_lower}'
        class_name = model_name
    
    try:
        module = importlib.import_module(module_path)
        return getattr(module, class_name)
    except (ImportError, AttributeError) as e:
        if use_base:
            raise ImportError(f"{model_name} base version not found at {module_path}.{class_name}: {e}")
        else:
            # Full 버전이 없으면 base로 폴백
            print(f"Full version not found ({module_path}.{class_name}), falling back to base version")
            return get_model_class(model_name, use_base=True)

ModelClass = get_model_class(test_model, use_base)
print(f"Loaded: {ModelClass.__name__}")

Loaded: Change3DBase


In [8]:
from configs import get_model_config

model_config = get_model_config(test_model)

optimizer = model_config['optimizer']
learning_rate = model_config['learning_rate']
weight_decay = model_config['weight_decay']
betas = model_config['betas']
eps = model_config['eps']
scheduler = model_config['scheduler']
momentum = model_config['momentum']

print(f"Model configurations:")
print(f"  Test_model: {test_model}")
print(f"  Optimizer: {optimizer}")
print(f"  Learning rate: {learning_rate}")
print(f"  Weight decay: {weight_decay}")
if betas:
    print(f"  Betas: {betas}")
if momentum:
    print(f"  Momentum: {momentum}")
print(f"  Scheduler: {scheduler}")


Model configurations:
  Test_model: Change3D
  Optimizer: adam
  Learning rate: 0.0002
  Weight decay: 0.0001
  Betas: (0.9, 0.999)
  Scheduler: poly


옵티마이저 생성 함수

In [9]:
def get_optimizer(model, config):
    """모델 설정에 따른 옵티마이저 생성"""
    if config['optimizer'] == 'adam':
        optimizer = torch.optim.Adam(
            model.parameters(), 
            lr=config['learning_rate'],
            betas=config['betas'],
            eps=config['eps'],
            weight_decay=config['weight_decay']
        )
    elif config['optimizer'] == 'adamw':
        optimizer = torch.optim.AdamW(
            model.parameters(),
            lr=config['learning_rate'],
            betas=config['betas'],
            eps=config['eps'],
            weight_decay=config['weight_decay']
        )
    elif config['optimizer'] == 'sgd':
        optimizer = torch.optim.SGD(
            model.parameters(),
            lr=config['learning_rate'],
            momentum=config['momentum'],
            weight_decay=config['weight_decay']
        )
    else:
        raise ValueError(f"Unknown optimizer: {config['optimizer']}")
    
    return optimizer

실험 폴더 설정

In [10]:
test_path = f"experiments/{test_dataset}"
test_dir = Path(f"{test_path}")
test_dir.mkdir(parents=True, exist_ok=True)

model_dir = Path(f"{test_path}/{test_model}")
model_dir.mkdir(parents=True, exist_ok=True)

checkpoint_dir = model_dir / "checkpoints"
checkpoint_dir.mkdir(exist_ok=True)

print(f"Experiment directory: {model_dir}")

Experiment directory: experiments/LEVIR-CD+/Change3D


Iteration 계산 
MAX_ITERATIONS = 100000을 기준으로 epoch 수 계산


In [11]:
dataset_path = Path(DATASET_ROOT) / test_dataset
print(dataset_path)
if dataset_path.exists():
    print(f"Dataset found: {dataset_path}")
    splits = ['train', 'val', 'test']
    dataset_info = {}
    
    for split in splits:
        split_path = dataset_path / split
        if split_path.exists():
            img_count = len(list((split_path / 't1').glob('*')))
            dataset_info[split] = img_count
            print(f"  {split}: {img_count} images")
    
    # Epoch 수 계산 (MAX_ITERATIONS 기준)
    if 'train' in dataset_info:
        train_samples = dataset_info['train']
        iterations_per_epoch = train_samples // BATCH_SIZE
        EPOCHS = MAX_ITERATIONS // iterations_per_epoch
        
        print(f"\nTraining iterations info:")
        print(f"  Train samples: {train_samples}")
        print(f"  Iterations per epoch: {iterations_per_epoch}")
        print(f"  Total epochs: {EPOCHS}")
        print(f"  Total iterations: {EPOCHS * iterations_per_epoch}")
else:
    print(f"Dataset not found: {dataset_path}")
    raise FileNotFoundError(f"Dataset {test_dataset} not found at {dataset_path}")

dataset/LEVIR-CD+
Dataset found: dataset/LEVIR-CD+
  train: 10192 images
  val: 1568 images
  test: 4000 images

Training iterations info:
  Train samples: 10192
  Iterations per epoch: 159
  Total epochs: 628
  Total iterations: 99852


데이터로더 생성

In [12]:
train_loader, val_loader, test_loader = create_dataloaders(
    root_dir=DATASET_ROOT,
    dataset_name=test_dataset,
    batch_size=BATCH_SIZE,
    num_workers=NUM_WORKERS
)


Loaded 10192 images from LEVIR-CD+/train
Loaded 1568 images from LEVIR-CD+/val
Loaded 4000 images from LEVIR-CD+/test


모델 학습 및 검증

In [13]:
model = ModelClass(num_classes=1).to(DEVICE)
criterion = get_loss_fn('bce_dice')

# 옵티마이저
if optimizer == 'adam':
    opt = torch.optim.Adam(
        model.parameters(),
        lr=learning_rate,
        weight_decay=weight_decay
    )
elif optimizer == 'adamw':
    opt = torch.optim.AdamW(
        model.parameters(),
        lr=learning_rate,
        weight_decay=weight_decay
    )

# 스케줄러
sched = None
if scheduler == 'cosine':
    sched = torch.optim.lr_scheduler.CosineAnnealingLR(opt, T_max=EPOCHS)

# %% Trainer 생성 및 학습
trainer = CDTrainer(
    model=model,
    optimizer=opt,
    criterion=criterion,
    device=DEVICE,
    checkpoint_dir=checkpoint_dir,
    scheduler=sched
)

# 학습 실행
trainer.train(
    train_loader=train_loader,
    val_loader=val_loader,
    epochs=EPOCHS,
    val_interval=10,
    save_interval=50
)

# %% 테스트 및 속도 측정
# 테스트
test_metrics = trainer.test(test_loader)

# 추론 속도
speed_metrics = trainer.measure_inference_speed(test_loader)

# 결과 저장
trainer.save_results(model_dir, test_model, test_dataset)

# %% 모델 파라미터 수
total_params = sum(p.numel() for p in model.parameters())
print(f"\nModel Parameters: {total_params:,}")

# 최종 결과
final_results = {
    'model': test_model,
    'base_version': use_base,
    'dataset': test_dataset,
    'test_metrics': test_metrics,
    'speed_metrics': speed_metrics,
    'parameters': total_params
}

with open(model_dir / 'final_results.json', 'w') as f:
    json.dump(final_results, f, indent=2)

print("\n✓ All completed!")


Starting Training
Total Epochs: 628
Iterations per epoch: 159
Total iterations: 99852



Overall Progress:   0%|          | 0/628 [00:00<?] 

  ├─Training:   0%|          | 0/159

Epoch    1/628 | Train: L=0.5277 F1=0.3000 | Time: 18.2s


  ├─Training:   0%|          | 0/159

...

  ├─Training:   0%|          | 0/159

Epoch    3/628 | Train: L=0.3854 F1=0.4431 | Time: 17.2s


  ├─Training:   0%|          | 0/159

Exception in thread Thread-8:
Traceback (most recent call last):
  File "/userHome/userhome4/kyoungmin/miniconda3/envs/cd_efficient/lib/python3.9/threading.py", line 980, in _bootstrap_inner
    self.run()
  File "/userHome/userhome4/kyoungmin/miniconda3/envs/cd_efficient/lib/python3.9/site-packages/ipykernel/ipkernel.py", line 772, in run_closure
    _threading_Thread_run(self)
  File "/userHome/userhome4/kyoungmin/miniconda3/envs/cd_efficient/lib/python3.9/threading.py", line 917, in run
    self._target(*self._args, **self._kwargs)
  File "/userHome/userhome4/kyoungmin/miniconda3/envs/cd_efficient/lib/python3.9/site-packages/torch/utils/data/_utils/pin_memory.py", line 54, in _pin_memory_loop
    do_one_step()
  File "/userHome/userhome4/kyoungmin/miniconda3/envs/cd_efficient/lib/python3.9/site-packages/torch/utils/data/_utils/pin_memory.py", line 31, in do_one_step
    r = in_queue.get(timeout=MP_STATUS_CHECK_INTERVAL)
  File "/userHome/userhome4/kyoungmin/miniconda3/envs/cd_effici

KeyboardInterrupt: 